In [65]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, confusion_matrix
import numpy as np

# Load the data with a different encoding
data = pd.read_csv("/Users/joonghochoi/Desktop/BNY Mellon/data/spam.csv", encoding='latin1')
data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
data.rename(columns={'v1': 'label', 'v2': 'text'}, inplace=True)
data['label'] = data['label'].apply(lambda label: 0 if label == 'ham' else 1)
data.head()


,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [66]:


# Split the data into training and testing sets (70/30 split)
train_df, test_df = train_test_split(data, test_size=0.3, random_state=42)

# Split the training data further into training and validation sets (40/30 split)
train_df, validation_df = train_test_split(train_df, test_size=0.3, random_state=42)

# Save the dataframes as CSV files
train_df.to_csv("/Users/joonghochoi/Desktop/BNY Mellon/data/train.csv", index=False)
validation_df.to_csv("/Users/joonghochoi/Desktop/BNY Mellon/data/validation.csv", index=False)
test_df.to_csv("/Users/joonghochoi/Desktop/BNY Mellon/data/test.csv", index=False)


In [67]:
train_df.head()

,label,text
1171,1,Got what it takes 2 take part in the WRC Rally...
4438,0,i want to grasp your pretty booty :)
4832,1,"New Mobiles from 2004, MUST GO! Txt: NOKIA to ..."
4972,0,Oops I was in the shower when u called. Hey a ...
1633,0,Yesterday its with me only . Now am going home.


In [68]:
train_df['text_length'] = train_df['text'].apply(lambda message: len(message))
train_df['no_of_chars'] = train_df['text'].apply(len)
train_df['no_of_words'] = train_df['text'].apply(lambda x: len(nltk.word_tokenize(x)))
train_df['no_of_sents'] = train_df['text'].apply(lambda x: len(nltk.sent_tokenize(x)))

# validation_df['text_length'] = validation_df['text'].apply(lambda message: len(message))

train_df.head()

,label,text,text_length,no_of_chars,no_of_words,no_of_sents
1171,1,Got what it takes 2 take part in the WRC Rally...,159,159,38,3
4438,0,i want to grasp your pretty booty :),36,36,9,1
4832,1,"New Mobiles from 2004, MUST GO! Txt: NOKIA to ...",153,153,29,4
4972,0,Oops I was in the shower when u called. Hey a ...,142,142,31,4
1633,0,Yesterday its with me only . Now am going home.,47,47,11,2


In [70]:
validation_df['text_length'] = validation_df['text'].apply(lambda message: len(message))
validation_df['no_of_chars'] = validation_df['text'].apply(len)
validation_df['no_of_words'] = validation_df['text'].apply(lambda x: len(nltk.word_tokenize(x)))
validation_df['no_of_sents'] = validation_df['text'].apply(lambda x: len(nltk.sent_tokenize(x)))

validation_df.head()

,label,text,text_length,no_of_chars,no_of_words,no_of_sents
3830,0,Let's pool our money together and buy a bunch ...,116,116,40,3
5531,0,Compliments to you. Was away from the system. ...,60,60,14,3
2526,0,Do u noe how 2 send files between 2 computers?,46,46,11,1
2482,0,Pansy! You've been living in a jungle for two ...,101,101,23,3
3423,1,Am new 2 club & dont fink we met yet Will B gr...,161,161,30,1


In [15]:
data=train_df

In [17]:
# spams = data['text'].iloc[(data['label'] == 1).values]
# hams = data['text'].iloc[(data['label'] == 0).values]
# print(spams[:10])
# print(hams[:10])

1171    Got what it takes 2 take part in the WRC Rally...
4832    New Mobiles from 2004, MUST GO! Txt: NOKIA to ...
1306    Enjoy the jamster videosound gold club with yo...
224     500 New Mobiles from 2004, MUST GO! Txt: NOKIA...
4614    Sunshine Quiz! Win a super Sony DVD recorder i...
1875    HMV BONUS SPECIAL 500 pounds of genuine HMV vo...
4384    Do you want a New Nokia 3510i Colour Phone Del...
4498    SMS SERVICES. for your inclusive text credits,...
2990    HOT LIVE FANTASIES call now 08707509020 Just 2...
672     Get ur 1st RINGTONE FREE NOW! Reply to this ms...
Name: text, dtype: object
4438                 i want to grasp your pretty booty :)
4972    Oops I was in the shower when u called. Hey a ...
1633      Yesterday its with me only . Now am going home.
5571                           Rofl. Its true to its name
2408            Dear where you will be when i reach there
4624                             I'm on the bus. Love you
3825    Goodmorning,my grandfather expired..so

In [72]:
binary_vectorizer = CountVectorizer(binary=True)
count_vectorizer = CountVectorizer()
tfidf_vectorizer = TfidfVectorizer()

In [73]:
def feature_extraction(df, test=False):
    #only use text
    if not test:
        tfidf_vectorizer.fit(df['text'])
    
    X = np.array(tfidf_vectorizer.transform(df['text']).todense())
    return X

In [74]:
X_train = feature_extraction(train_df)
y_train = train_df['label'].values

X_valid = feature_extraction(validation_df, test=True)
y_valid = validation_df['label'].values

In [75]:
train_df.head(2)

,label,text,text_length,no_of_chars,no_of_words,no_of_sents
1171,1,Got what it takes 2 take part in the WRC Rally...,159,159,38,3
4438,0,i want to grasp your pretty booty :),36,36,9,1


In [76]:
train_df['label'].values

array([1, 0, 1, ..., 0, 0, 1])

In [77]:
clfs = {
    'mnb': MultinomialNB(),
    'gnb': GaussianNB(),
    'bnb':BernoulliNB()

}

In [78]:
f1_scores = dict()
for clf_name in clfs:
    print(clf_name)
    clf = clfs[clf_name]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_valid)
    f1_scores[clf_name] = f1_score(y_pred, y_valid)

mnb
gnb
bnb


In [79]:
f1_scores


{'mnb': 0.7520661157024793,
 'gnb': 0.7287234042553191,
 'bnb': 0.8773234200743495}

In [80]:
def feature_extraction(df, test=False):
    #use text and length
    if not test:
        count_vectorizer.fit(df['text'])
    
    X = np.array(count_vectorizer.transform(df['text']).todense())
    X = np.concatenate((X, df['text_length'].values.reshape(-1, 1)), axis=1)
    return X

In [81]:
X_train = feature_extraction(train_df)
y_train = train_df['label'].values

X_valid = feature_extraction(validation_df, test=True)
y_valid = validation_df['label'].values

In [82]:
clfs = {
    'mnb': MultinomialNB(),
    'gnb': GaussianNB(),
    'bnb':BernoulliNB()

}
f1_scores = dict()
for clf_name in clfs:
    print(clf_name)
    clf = clfs[clf_name]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_valid)
    f1_scores[clf_name] = f1_score(y_pred, y_valid)

mnb
gnb
bnb


In [83]:
f1_scores
#using text length helps. mnb is the best

{'mnb': 0.9351535836177475,
 'gnb': 0.7421052631578946,
 'bnb': 0.8773234200743495}

In [86]:
def feature_extraction(df, test=False):
    #use text and length
    if not test:
        count_vectorizer.fit(df['text'])
    
    X = np.array(count_vectorizer.transform(df['text']).todense())
    X = np.concatenate((X, df['no_of_chars'].values.reshape(-1, 1)), axis=1)
    X = np.concatenate((X, df['no_of_words'].values.reshape(-1, 1)), axis=1)
    X = np.concatenate((X, df['no_of_sents'].values.reshape(-1, 1)), axis=1)
    return X


In [85]:
X_train = feature_extraction(train_df)
y_train = train_df['label'].values

X_valid = feature_extraction(validation_df, test=True)
y_valid = validation_df['label'].values
clfs = {
    'mnb': MultinomialNB(),
    'gnb': GaussianNB(),
    'bnb':BernoulliNB()

}
f1_scores = dict()
for clf_name in clfs:
    print(clf_name)
    clf = clfs[clf_name]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_valid)
    f1_scores[clf_name] = f1_score(y_pred, y_valid)
f1_scores
#using text length helps. mnb is the best

mnb
gnb
bnb


{'mnb': 0.9379310344827586,
 'gnb': 0.7421052631578946,
 'bnb': 0.8773234200743495}

In [87]:
X_train = feature_extraction(train_df)
X_valid = feature_extraction(validation_df, test=True)

In [88]:
alpha_values = [i * 0.1 for i in range(11)]
max_f1_score = float('-inf')
best_alpha = None
for alpha in alpha_values:
    clf = MultinomialNB(alpha=alpha)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_valid)
    current_f1_score = f1_score(y_pred, y_valid)
    if current_f1_score > max_f1_score:
        max_f1_score = current_f1_score
        best_alpha = alpha

/Users/joonghochoi/miniconda3/envs/sweAI/lib/python3.9/site-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(


In [89]:
print('Best alpha: {0}'.format(best_alpha))
print('Best f1-score: {0}'.format(max_f1_score))

Best alpha: 0.1
Best f1-score: 0.9568106312292359


In [90]:
test_data=pd.read_csv("/Users/joonghochoi/Desktop/BNY Mellon/data/test.csv")
test_data

,label,text
0,0,"Funny fact Nobody teaches volcanoes 2 erupt, t..."
1,0,I sent my scores to sophas and i had to do sec...
2,1,We know someone who you know that fancies you....
3,0,Only if you promise your getting out as SOON a...
4,1,Congratulations ur awarded either å£500 of CD ...
...,...,...
1667,0,Congrats kano..whr s the treat maga?
1668,0,"Say this slowly.? GOD,I LOVE YOU &amp; I NEED ..."
1669,0,You are gorgeous! keep those pix cumming :) th...
1670,0,Thats cool! Sometimes slow and gentle. Sonetim...


In [92]:
test_data['text_length'] = test_data['text'].apply(lambda message: len(message))
test_data['no_of_chars'] = test_data['text'].apply(len)
test_data['no_of_words'] = test_data['text'].apply(lambda x: len(nltk.word_tokenize(x)))
test_data['no_of_sents'] = test_data['text'].apply(lambda x: len(nltk.sent_tokenize(x)))


In [101]:
clf = MultinomialNB(alpha=0.1)
X_train = feature_extraction(train_df)
y_train = train_df['label'].values
X_test = feature_extraction(test_data, test=True)
y_test = test_data['label'].values

In [102]:
clf.fit(X_train, y_train)


MultinomialNB(alpha=0.1)

In [104]:
y_pred = clf.predict(X_test)
print('F1-score: {0}'.format(f1_score(y_pred, y_test)))
print('Confusion matrix:')
confusion_matrix(y_pred, y_test)

F1-score: 0.9241706161137442
Confusion matrix:


array([[1445,   24],
       [   8,  195]])